# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

In [3]:
df1 = pd.DataFrame(df1)

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [4]:
df1 = df1.drop_duplicates()

In [5]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [6]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [7]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [8]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [9]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [10]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [11]:
from datetime import timedelta, datetime

In [12]:
df1['first_screening_date']

0      26/10/2023
1      15/11/2023
2      08/11/2023
3      16/11/2023
4      16/11/2023
          ...    
940    18/01/2024
941    16/09/2021
942    25/01/2024
943    01/02/2024
944    08/02/2024
Name: first_screening_date, Length: 860, dtype: object

In [13]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

C:\Users\Franci\AppData\Local\Temp\ipykernel_26648\2741308956.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')


In [14]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [15]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [16]:
df1['screening_days']

0      106
1       86
2       93
3       85
4       85
      ... 
940     22
941    876
942     15
943      8
944       
Name: screening_days, Length: 860, dtype: object

In [17]:
#df1['screening_days'] = df1['screening_days'].str.strip()
df1['screening_days'] = df1['screening_days'].replace('', '0')
df1['screening_days'] = df1['screening_days'].astype(int) + 1

In [18]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [19]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [20]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [21]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,107,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,87,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,94,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,86,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,7,Dune,2021-09-16,USA,Warner Bros Italia S.P.A.,15255.43,2144,7374477.30,1028210,2024-02-09,877,2024-02-08,Thursday,7.12
942,8,Dieci Minuti,2024-01-25,ITA,Vision Distribution,15174.38,2426,735675.73,110559,2024-02-09,16,2024-02-08,Thursday,6.25
943,9,Argylle - La Super Spia (Argylle),2024-02-01,USA,Universal S.R.L.,14815.34,2184,497243.10,66835,2024-02-09,9,2024-02-08,Thursday,6.78
944,10,Il Colore Viola (The Color Purple),2024-02-08,USA,Warner Bros Italia S.P.A.,12599.06,2081,12629.06,2084,2024-02-09,1,2024-02-08,Thursday,6.05


---

## Export as .csv

In [22]:
import os

In [23]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [24]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
pd.set_option('display.max_rows', 8)

In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 860 entries, 0 to 944
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            860 non-null    int32         
 1   title                 860 non-null    object        
 2   first_screening_date  856 non-null    object        
 3   nation                860 non-null    object        
 4   distribution          860 non-null    object        
 5   daily_takings         860 non-null    float64       
 6   daily_attendance      860 non-null    int32         
 7   total_takings         860 non-null    float64       
 8   total_attendance      860 non-null    int32         
 9   date_pulled           860 non-null    object        
 10  screening_days        860 non-null    int32         
 11  date                  860 non-null    datetime64[ns]
 12  day_of_week           860 non-null    object        
 13  avg_ticket_price      860

In [28]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,date,avg_ticket_price
count,860.00,860.00,860.00,860.00,860.00,860.00,860,860.00
mean,5.50,165842.20,23038.15,5058021.30,720655.73,61.60,2023-12-27 15:04:11.162790656,6.96
min,1.00,6272.70,864.00,11652.78,1470.00,1.00,2023-11-16 00:00:00,3.70
25%,3.00,34487.29,5227.75,525798.26,77652.00,37.00,2023-12-07 00:00:00,6.42
50%,5.50,84202.61,12254.50,1869637.52,268732.50,58.00,2023-12-26 12:00:00,6.97
75%,8.00,207850.12,29121.25,4516631.34,647718.00,79.00,2024-01-17 00:00:00,7.39
max,10.00,1544231.00,211764.00,36064082.56,5296807.00,877.00,2024-02-08 00:00:00,21.18
std,2.87,212012.81,28443.92,8587668.85,1254902.09,40.41,NaN,1.16


In [29]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    4
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [30]:
df1.nunique()

daily_rank               10
title                    69
first_screening_date     40
nation                    9
                       ... 
screening_days           40
date                     84
day_of_week               7
avg_ticket_price        840
Length: 14, dtype: int64

In [31]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.57,-0.58,-0.12,-0.12,0.08,-0.07
daily_takings,-0.57,1.00,1.00,0.14,0.13,-0.03,0.21
daily_attendance,-0.58,1.00,1.00,0.16,0.15,-0.02,0.17
total_takings,-0.12,0.14,0.16,1.00,1.00,0.36,-0.15
total_attendance,-0.12,0.13,0.15,1.00,1.00,0.36,-0.16
screening_days,0.08,-0.03,-0.02,0.36,0.36,1.00,-0.11
avg_ticket_price,-0.07,0.21,0.17,-0.15,-0.16,-0.11,1.00


In [32]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,4.29,272838.27,40336.83,30313309.75,4439977.77,107.00,6.40
Wonka,3.85,368953.34,48842.15,9842301.36,1311072.30,58.00,7.32
Dune,7.00,15255.43,2144.00,7374477.30,1028210.00,877.00,7.12
Wish,3.93,395833.99,55969.18,5304161.62,756048.46,51.00,6.92
...,...,...,...,...,...,...,...
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,143.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,56.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,75.00,9.62
Il Colore Viola (The Color Purple),10.00,12599.06,2081.00,12629.06,2084.00,1.00,6.05


In [33]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,4.29,272838.27,40336.83,30313309.75,4439977.77,107.00,6.40
Wonka,3.85,368953.34,48842.15,9842301.36,1311072.30,58.00,7.32
Dune,7.00,15255.43,2144.00,7374477.30,1028210.00,877.00,7.12
Wish,3.93,395833.99,55969.18,5304161.62,756048.46,51.00,6.92
...,...,...,...,...,...,...,...
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,143.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,56.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,75.00,9.62
Il Colore Viola (The Color Purple),10.00,12599.06,2081.00,12629.06,2084.00,1.00,6.05
